# example 1: provide data and use existing model

In [17]:
!textattack attack --recipe textfooler --dataset-from-file "test_data.py" --model bert-base-uncased-mr

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Loading pre-trained model from HuggingFace model repository: textattack/bert-base-uncased-rotten-tomatoes
Downloading: 100%|██████████████████████████████| 487/487 [00:00<00:00, 202kB/s]
Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding_type):  paragramcf
  )


# example2:provide model

In [57]:
!textattack attack --model-from-file model1.py --recipe textfooler --num-examples 1000

textattack: Loading model and tokenizer from file: model1.py
textattack: Loading module from `model1.py`.
Traceback (most recent call last):
  File "/home/qc/anaconda3/bin/textattack", line 8, in <module>
    sys.exit(main())
  File "/home/qc/anaconda3/lib/python3.7/site-packages/textattack/commands/textattack_cli.py", line 42, in main
    args.func.run(args)
  File "/home/qc/anaconda3/lib/python3.7/site-packages/textattack/commands/attack/attack_command.py", line 42, in run
    run_single_threaded(args)
  File "/home/qc/anaconda3/lib/python3.7/site-packages/textattack/commands/attack/run_attack_single_threaded.py", line 47, in run
    attack = parse_attack_from_args(args)
  File "/home/qc/anaconda3/lib/python3.7/site-packages/textattack/commands/attack/attack_args_helpers.py", line 190, in parse_attack_from_args
    model = parse_model_from_args(args)
  File "/home/qc/anaconda3/lib/python3.7/site-packages/textattack/commands/attack/attack_args_helpers.py", line 272, in parse_model_fro

## this is torch syntax, wrapper needed!!

# example 3: sklearn model

In [38]:
!textattack attack --model-from-file model2.py --recipe deepwordbug --num-examples 10 --dataset-from-nlp imdb

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Loading model and tokenizer from file: model2.py
textattack: Loading module from `model2.py`.
textattack: Unknown if model of class <class 'temp_1596169849.9316492.SklearnModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_dista

# example 4: bypass command line api

In [47]:
import textattack
import pickle
import pandas as pd
import torch

In [69]:
from collections import deque
num_remaining_attacks = 1
worklist = deque(range(0, num_remaining_attacks))
worklist_tail = worklist[-1]

In [45]:
# load model

In [70]:
class Bunch(object):
    def __init__(self, adict):
        self.__dict__.update(adict)


class SklearnTokenizer:
    def __init__(self, tokenizer_path):
        # load tokenizer from file
        self.tokenizer = pickle.load(open(tokenizer_path, "rb"))

    def batch_encode(self, text_list):
        if isinstance(text_list[0], tuple):  # unroll tuples passed in by TextAttack
            text_list = [x[0] for x in text_list]
        encoded_text_matrix = self.tokenizer.transform(text_list)
        return torch.tensor(encoded_text_matrix.toarray())

    def encode(self, text):
        return self.batch_encode([text])


class SklearnModelWrapper:
    def __init__(self, model_path, feature_names):
        self.model = pickle.load(open(model_path, 'rb'))
        self.feature_names = feature_names

        # dummy attribute that will also not be necessary in the future
        def dummy_params():
            yield Bunch({'device': torch.device('cuda')})

        setattr(self.model, 'parameters', dummy_params)

    def __call__(self, tokenized_text_input):
        tokenized_text_df = pd.DataFrame(tokenized_text_input, columns=self.feature_names)
        return torch.tensor(self.model.predict_proba(tokenized_text_df))

    def to(self, *args):
        # dummy method that will not be necessary in the future
        return self


tokenizer = SklearnTokenizer('/home/qc/work/imdb_adversarial_attack/tf_idf_vectorizor2.pkl')
feature_names = tokenizer.tokenizer.get_feature_names()
model = SklearnModelWrapper('/home/qc/work/imdb_adversarial_attack/tf_idf_logistic_reg.pkl', feature_names)


model = model.to(textattack.shared.utils.device)
setattr(model, "tokenizer", tokenizer)

In [71]:
# define attack

In [72]:
from textattack.attack_recipes import DeepWordBugGao2018

In [73]:
recipe = DeepWordBugGao2018(model)

textattack: Unknown if model of class <class '__main__.SklearnModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [74]:
recipe

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)

In [75]:
recipe.goal_function.model_batch_size = 32
recipe.goal_function.model_cache_size = 262144
recipe.constraint_cache_size = 262144

In [76]:
# get data set
_df = pd.read_csv('test_data.csv', header=None)
_df.columns = ['label', 'text']
dataset = _df[['text', 'label']].values.tolist()

In [64]:
# run things

In [78]:
results = []

In [79]:
for result in recipe.attack_dataset(dataset, indices=worklist):
    results.append(result)

In [106]:
print(results[0].__str__("ansi"))

1 (52%) --> 0 (51%)

Wall St. Bears Claw Back Into the Black ( Reuters ) . Reuters - Short - sellers , Wall Street 's dwindling band of ultra - cynics , are seeing green again .

Wall St. Beajrs Claw Back Into the Black ( Reuters ) . Reuters - Short - sellers , Wall Street 's dwindling band of ultra - cynics , are seeing green again .


In [90]:
results[0].original_result.ground_truth_output

1

In [95]:
results[0].original_result.score

0.48340441672921686

In [108]:
results[0].original_result.attacked_text

<AttackedText "Wall St. Bears Claw Back Into the Black ( Reuters ) . Reuters - Short - sellers , Wall Street 's dwindling band of ultra - cynics , are seeing green again .">

In [98]:
results[0].perturbed_result.num_queries

29

In [93]:
results[0].perturbed_result.output

0

In [94]:
results[0].perturbed_result.score

0.5102398402251678

In [112]:
results[0].perturbed_result.attacked_text.all_words_diff(results[0].original_result.attacked_text)

{2}